Objective 1b - build a working model of multiple rounds of Blackjack, with betting functionality

There are a number of functionalities that I would like to build into the module in another script to start answering some meatier questions.

A simple betting module that can in its most simplest iteration specify a player's betting behaviour.
It must be able to accurately compute amounts paid out at the end of each hand.
It must be able to track amounts paid out by the dealer, and amounts won by the player historically.

Amend the existing modules to simulate the effect of multiple rounds being played in one "shoe", as is done in casinos, and this is crucial to assess the effects of card counting as a later objective. 

Skeleton module completed. Functionality:

* Module can encode initial capital stock, simple rule-based player betting behaviour at the beginning. 
* Module can simulate multiple hands being played.
* Module can compute payoffs and losses at the end of each hand, with caveats, and store the evolution of the player's capital stock, which can subsequently accessed for statistical analysis.
* Module can simulate the effects of reshuffling close to the end of the shoe. Frequency of reshuffling is controlled by a user defined threshold parameter that represents the cards remaining as a proportion of the total number of cards that the casino uses at the beginning of the shoe.

In [548]:
# Import modules

import random
import timeit
from collections import Counter

In [607]:
def getcards(decks=1):
    """Brings out a number of sealed decks of cards in their original order to the virutal gaming table.

    Creates a dictionary tracking the name of each card, its quantity, and its value
    
    Keyword argument:
        decks {int} -- the number of 52-card decks that the casino is using (default 1)
    
    Returns:
        dictionary{keys: values} -- dictionary object where keys are the card name (e.g. 'K' for King )
                                    and where the values are a list object containing the card's quantity and its value.
                                    
    The value for the ace contains three elements, as aces can further take on the value of 1 or 11, depending on 
    the context of the hand. No distinction is made between card suits, i.e. spades, hearts, clubs, diamonds.
    """
    
    deck = {}
    total_cards = decks * 52
    card_quantity = int(total_cards / 13)
    
    deck['A'] = [card_quantity, 1, 11]

    for card in range(2, 11):
        deck[str(card)] = [card_quantity, card]
    
    for card in "JQK":
        deck[str(card)] = [card_quantity, 10]
        
    return deck, total_cards

deck, total_cards = getcards(decks=1)

In [608]:
def shuffler(deck, total_cards):
    
    """Shuffles the decks of cards, and places them inside the virtual gaming table card-shoe."""
    
    shuffled_shoe = []
    
    while len(shuffled_shoe) < total_cards:
        draw = random.choice(list(deck))
        if deck[draw][0] == 0:
            pass
        else:
            deck[draw][0] -= 1
            shuffled_shoe.append(draw)
            
    return shuffled_shoe

shuffled_shoe = shuffler(deck, total_cards)

In [609]:
print(shuffled_shoe)

['J', '3', '6', '2', '10', 'Q', '3', '5', 'K', '7', '4', '7', '9', '7', '3', 'K', '8', 'J', '9', '2', 'Q', '5', 'Q', '5', '7', '10', 'A', '3', 'K', 'J', '2', '2', 'K', '8', 'A', 'A', '5', 'Q', 'A', '8', '10', '10', '8', 'J', '6', '9', '9', '6', '4', '4', '4', '6']


In [595]:
# Set initial capital and player betting behaviour

initial_capital = 500
capital_stock = [initial_capital]
bet_per_round = 5

In [676]:
def openinghand():
    
    """Deals two cards to the player, and one to the dealer from the shuffled shoe.
    
    Cards are taken sequentially from the shuffled card shoe, beginning with the last element of the shuffled shoe list object
    and working in reverse order."""
    
    player = []
    dealer = []

    player.append(shuffled_shoe.pop())
    player.append(shuffled_shoe.pop())
    
    dealer.append(shuffled_shoe.pop())
    
    if ('A' in player) and (('10' in player) or ('J' in player) or ('Q' in player) or ('K' in player)):
        print("Blackjack, house pays out at 3:2")
    else:
        print("No Blackjack this time")
        
    return player, dealer

player, dealer = openinghand()

def playerturn():
    
    """Evaluates the player's score, and keeps drawing cards until the player has a score of more than 18"""
    
    cardvalues = []
    for card in player:
        cardvalues.append(deck[card][1])

    while sum(cardvalues) < 18:
        drawcard = shuffled_shoe.pop()
        player.append(drawcard)
        cardvalues.append(deck[drawcard][1])
        print("Decided to hit, as I'm a risk lover and under 18")
        
    if sum(cardvalues) > 21:
        print("...I've gone bust. Why is my lucky charm that I got from the Amazonian shaman not working?")
    else:
        print("Decided to stick")
        
    print("Player Score:" + str(sum(cardvalues)))
    print(player)
        
    return player, sum(cardvalues)

player, playerscore = playerturn() 

def dealerturn():
    
    """Evaluates the dealer's score, and keeps drawing cards until dealer has a score of 17 or more."""
    
    cardvalues = []
    for card in dealer:
        cardvalues.append(deck[card][1])
        
    while sum(cardvalues) <= 16:
        drawcard = shuffled_shoe.pop()
        dealer.append(drawcard)
        cardvalues.append(deck[drawcard][1])
        print("Dealer draws a card as under or equal to 16")
    
    if sum(cardvalues) > 21:
        print("Dealer goes bust")
    else:
        print("Dealer sticks on 17 or over")
    
    print("Dealer Score:" + str(sum(cardvalues)))
    print(dealer)
    
    return dealer, sum(cardvalues)

dealer, dealerscore = dealerturn()

def handoutcome():
    
    """Compares player's and dealer's scores and encodes the outcome.
    
    Can be extended if analysis required tracking the evolution of this variable."""

    if playerscore > 21:
        if dealerscore > 21:
            hand_outcome = 0
            print("Player busts. Dealer also busts. But the 'edge' means that player loses the hand.")
        else:
            hand_outcome = 0
            print("Player busts. Dealer wins the hand.")
    else:
        if dealerscore > 21:
            hand_outcome = 1
            print("Dealer busts. Player wins the hand")
        else:
            if dealerscore > playerscore:
                hand_outcome = 0
                print("Dealer has higher score. Dealer wins the hand")
            elif dealerscore < playerscore:
                hand_outcome = 1
                print("Player has higher score. Player wins the hand")
            else:
                hand_outcome = 2
                print("Both dealer and player have the same score, the outcome of the hand is a draw")
    
    return hand_outcome

hand_outcome = handoutcome()

def payout_loss(hand_outcome):
    
    """Uses the outcome to calculate payoffs/losses, and updates player's capital stock accordingly."""
    
    if hand_outcome == 1:
        new_capital = capital_stock[-1] + bet_per_round
        capital_stock.append(new_capital)
        print("House pays out " + str(bet_per_round) + " pounds")
    elif hand_outcome == 0:
        new_capital = capital_stock[-1] - bet_per_round
        capital_stock.append(new_capital)
        print("Player loses " + str(bet_per_round) + " pounds")
    else:
        print("Draw, no change to player's initial capital")
        
    print("Player now has" + " " + str(capital_stock[-1]) + " " + "pounds remaining.")
        
    return capital_stock

capital_stock = payout_loss(hand_outcome)

def replenish_shoe(threshold, total_cards, deck, shuffled_shoe):
    
    """Checks whether or not  we are near the end of a shoe. If so, discards the remaining cards in the current shoe,
    collects them together with remaining cards that were discarded in previous hands, reshuffles all of them togther,
    and finally replenishes the shoe."""
    
    fraction_cards_remaining = len(shuffled_shoe) / total_cards
    
    if fraction_cards_remaining < threshold:
        shuffled_shoe.clear
        deck, total_cards = getcards(decks=1)
        shuffled_shoe = shuffler(deck, total_cards)
        print("As we are reaching near the end of the shoe, we will reshuffle")
    else:
        print("No need for reshuffling yet, as card threshold not reached")

    return shuffled_shoe

shuffled_shoe = replenish_shoe(0.1, total_cards, deck, shuffled_shoe)

No Blackjack this time
Decided to hit, as I'm a risk lover and under 18
Decided to hit, as I'm a risk lover and under 18
...I've gone bust. Why is my lucky charm that I got from the Amazonian shaman not working?
Player Score:24
['6', '3', '5', 'K']
Dealer draws a card as under or equal to 16
Dealer draws a card as under or equal to 16
Dealer sticks on 17 or over
Dealer Score:20
['Q', '3', '7']
Player busts. Dealer wins the hand.
Player loses 5 pounds
Player now has 490 pounds remaining.
No need for reshuffling yet, as card threshold not reached


In [677]:
print(capital_stock)

[500, 505, 510, 515, 510, 505, 500, 495, 490, 485, 490, 495, 490, 485, 490, 485, 480, 475, 470, 475, 480, 475, 470, 475, 480, 485, 490, 495, 500, 505, 500, 495, 490]


In [611]:
# Worry about efficiency later
# O/S - basic strategy
# O/S - hard vs soft hands and dual ace value
# O/S - player double, split, 

# Player blackjack automatically terminates the round

# Have to code in the ability to simulate multiple iterations
# Have to address the issue of the shuffled shoe running out of cards. Can amend this by coding a replenish rule that 
# Is now the time to separate function definition and function calls so that I can put them through a simulator that runs it
# many times?

In [612]:
# Betting and payout modules
# There might be an issue with your payout_loss() function - not storing post-hand capital changes appropriately.
# Takes the outcome of the hand and computes payoffs or losses. Adjusts the player's capital stock depending on the outcome
# of the hand. 

# * Now have rough knowledge of referencing variables outside a function, and the distinction between global and local variables
# together with namespaces. Area for further reading - Python documentation would be best.
# I know that modifying (assigning) a global variable outside a function inside a local context yields an unbound error using equals
# But looking at this module, does this mean that you can still call methods on global variables? i.e. use append

def payout_loss(hand_outcome):
    
    if hand_outcome == 1:
        new_capital = capital_stock[-1] + bet_per_round
        capital_stock.append(new_capital)
        print("House pays out " + str(bet_per_round) + " pounds")
    elif hand_outcome == 0:
        new_capital = capital_stock[-1] - bet_per_round
        capital_stock.append(new_capital)
        print("Player loses " + str(bet_per_round) + " pounds")
    else:
        print("Draw, no change to player's initial capital")
        
    print("Player now has" + " " + str(capital_stock[-1]) + " " + "pounds remaining.")
        
    return capital_stock

capital_stock = payout_loss(hand_outcome)

House pays out 5 pounds
Player now has 515 pounds remaining.


In [613]:
# End of shoe - replenish the shuffled shoe according to a shuffle-frequency parameter; a user-defined parameter that implicitly
# determines how quickly a shoe finishes, cards are replenished and reshuffling occurs. This parameter is specified
# as the percentage of cards remaining in the shoe, and when the number of cards in the shoe falls below this percentage,
# replenishment occurs. Make concise explanation later but for now it suffices.

# Amend the commentary of getcards() - it is not simulating getting cards from the back room , but also corresponds to
# replenshing it. 

# Payoff_loss() will continue to use the last hand outcome and make adjsutments to the player's capital stock even after
# there are no longer any cards to be dealt due to an empty card shoe.

# Getting the sense that this will be a perfect opportunity to learn about classes,namespaces, local and global variables

# Opted to include deck, shuffled_shoe and total_cards as arguments to this function rather than declaring them within
# the local context of the function as global variables


def replenish_shoe(threshold, total_cards, deck, shuffled_shoe):
    
    fraction_cards_remaining = len(shuffled_shoe) / total_cards
    
    if fraction_cards_remaining < threshold:
        shuffled_shoe.clear
        deck, total_cards = getcards(decks=1)
        shuffled_shoe = shuffler(deck, total_cards)
        print("As we are reaching near the end of the shoe, we will reshuffle")
    else:
        print("No need for reshuffling yet, as card threshold not reached")

    return shuffled_shoe

shuffled_shoe = replenish_shoe(0.1, total_cards, deck, shuffled_shoe)

No need for reshuffling yet, as card threshold not reached


In [614]:
capital_stock

[500, 505, 510, 515]

In [465]:
# Python is returning an error when I use this code module "local variable 'total_cards' referenced before assignment".
# I don't understand, but don't want this to disrupt the flow so I will come back to it later. For now deactivate it,
# find a quick workaround and dig a little later. It's either to do with namespacing or local and global variables.

def replenish_shoe(threshold):
    
    fraction_cards_remaining = len(shuffled_shoe) / total_cards
    
    if fraction_cards_remaining < threshold:
        shuffled_shoe.clear()
        deck, total_cards = getcards(decks=1)
        print("As we are reaching near the end of the shoe, we will reshuffle")
        
    else:
        return shuffled_shoe

In [466]:
replenish_shoe(0.1)

UnboundLocalError: local variable 'total_cards' referenced before assignment

In [440]:
% reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y
